In [292]:
import json
import numpy as np
import os
import subprocess
import glob
from soma import aims

from sulci.registration.spam import spam_register

import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma.aimsalgo import MorphoGreyLevel_S16

# Global static variables
_AIMS_BINARY_ONE = 32767
_dilation = 0
_threshold = 3
_dilation_final = 0
_threshold_final = 3

# launching anatomist
a = ana.Anatomist()

In [293]:
def dilate(mask, radius=_dilation):
    """Makes a dilation radius _dilation, in mm
    """
    arr = mask.np
    # Binarization of mask
    arr[arr < 1] = 0
    if radius > 0:
        arr[arr >= 1] = _AIMS_BINARY_ONE
        # Dilates initial volume of 10 mm
        morpho = MorphoGreyLevel_S16()
        dilate = morpho.doDilation(mask, radius)
        arr_dilate = dilate.np
        arr_dilate[arr_dilate >= 1] = 1
        return dilate
    else:
        arr[arr >= 1] = 1
        return mask

In [294]:
spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/Sc.Cal.-S.Li._left.nii.gz'
skel_path = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L'
skel_files = glob.glob(f'{skel_path}/*.nii.gz')
skel_files[:5]

['/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_hades.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_sujet04.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_icbm200T.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_shiva.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_hyperion.nii.gz']

In [295]:
# Reads initial spam volume
spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")

In [296]:
print(np.unique(spam_vol.np))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.]


In [297]:
def do_masking_dilation(spam_vol, skel_vol, dilation, threshold, do_binarization):
   
    spam_vol = aims.Volume_FLOAT(spam_vol)
    skel_vol = aims.Volume_S16(skel_vol)
    
    # Do binarization for registration
    if do_binarization:
        skel_vol.np[:] = (skel_vol.np > 0).astype(np.int16)

    # Makes binarization and dilation on spam
    mask_result = aims.Volume(spam_vol.getSize(), 'S16')
    mask_result.copyHeaderFrom(spam_vol.header())
    mask_result.np[:] = spam_vol.np
    print("before threshold", np.unique(mask_result.np, return_counts=True)) 
    mask_result.np[mask_result.np <= threshold] = 0.
    mask_result.np[:] = dilate(mask_result, dilation).np
    print("after threshold", np.unique(mask_result.np, return_counts=True))
    
    # Do the actual masking
    skel_vol.np[mask_result.np <= 0] = 0
    
    return skel_vol

In [298]:
def realign(spam_vol: aims.Volume_FLOAT, skel_vol: aims.Volume_S16):
    """Realigns skeleton mask to spam
    
    skel_f is a file name of skeleton file"""
    
    spam_vol = aims.Volume_FLOAT(spam_vol)
    skel_vol = aims.Volume_S16(skel_vol)
    
    # Masks with first dilation and threshold
    skel_vol_before = do_masking_dilation(spam_vol, skel_vol, _dilation, _threshold, True)
    aims.write(skel_vol_before, "/tmp/skel_before.nii.gz")
    print(np.unique(spam_vol.np))
    
    # Makes realignment
    out_tr = spam_register(spam_vol/np.max(spam_vol.np),
                        skel_vol_before,
                        do_mask=False,
                        R_angle_var=np.pi / 8,
                        t_var=10.,
                        verbose=False,
                        in_log=False,
                        calibrate_distrib=30)
    aims.write(out_tr, '/tmp/transform.trm')
    print(out_tr.np)
    
    # Masks with final dilation and threshold
    skel_vol = do_masking_dilation(spam_vol, skel_vol, _dilation_final, _threshold_final, False)
    aims.write(skel_vol, "/tmp/skel_final_before.nii.gz")
    
    # Applies the realignment
    subprocess.check_call(f"AimsApplyTransform -i /tmp/skel_final_before.nii.gz -o /tmp/skel_final_realigned.nii.gz -m /tmp/transform.trm", shell=True)
    
    # loads realigned file:
    before = aims.read("/tmp/skel_final_before.nii.gz")
    after = aims.read("/tmp/skel_final_realigned.nii.gz")
    
    return before, after

In [299]:
before_all = aims.Volume(spam_vol.getSize(), 'S16')
before_all.copyHeaderFrom(spam_vol.header())
after_all = aims.Volume(spam_vol.getSize(), 'S16')
after_all.copyHeaderFrom(spam_vol.header())
list_after = []
for skel_f in skel_files[:1]:
    skel_vol = aims.read(skel_f)
    b, r = realign(spam_vol, skel_vol)
    before = aims.Volume(spam_vol.getSize(), 'S16')
    before.copyHeaderFrom(spam_vol.header())
    before += b
    after = aims.Volume(spam_vol.getSize(), 'S16')
    after.copyHeaderFrom(spam_vol.header())
    after += r
    
    before_all += before
    after_all += after
    list_after.append(after)

before threshold (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34], dtype=int16), array([1046227,    1234,     641,     410,     296,     238,     209,
           184,     165,     120,     129,     104,      96,      85,
            75,      69,      53,      52,      49,      27,      25,
            22,      22,      11,      17,      15,      10,       9,
             8,       7,       6,       3,       2,       2,       2]))
after threshold (array([0, 1], dtype=int16), array([1048512,    2112]))
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.]
[[ 1.0000000e+00 -1.1917869e-11 -2.2937838e-08  0.0000000e+00]
 [ 1.1918176e-11  1.0000000e+00  1.5240136e-08  0.0000000e+00]
 [ 2.2937838e-08 -1.5240136e-08  1.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loading direct transformations
Output dimensions: 96, 114, 96
Output voxel size: 2, 2, 2 mm
Resampling carto_volume of S16...   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 %


In [300]:
np.unique(before_all.np)

array([0, 1], dtype=int16)

In [301]:
np.unique(after_all.np)

array([0, 1], dtype=int16)

In [302]:
diff_all = after_all - before_all

In [303]:
diff_all

In [304]:
# Visualization
spam = a.loadObject(spam_file)
before_a = a.toAObject(before_all)
diff_a = a.toAObject(diff_all)
before_a.setPalette("Blues")
diff_a.setPalette("bwr")
# spam_before = a.toAObject(before_all)
# spam_before.setPalette("Blues")
# spam_after = a.toAObject(after_all)
# spam_after.setPalette("Reds")
w = a.createWindow('Sagittal')
w.addObjects(spam)
w.addObjects(before_a)
w.addObjects(diff_a)
# w.addObjects(spam_before)
# w.addObjects(spam_after)

observable 0x614142469120(N9anatomist7AVolumeIsEE) could not be removed from observer 0x614144b03dd8 (N9anatomist8Fusion2DE)
observable 0x61414239e220(N9anatomist7AVolumeIsEE) could not be removed from observer 0x6141416dfcb8 (N9anatomist8Fusion2DE)


In [305]:
# spam = a.loadObject(spam_file)
# spam.setPalette("Blues")
# list_after_a = [a.toAObject(after) for after in list_after]
# for after in list_after_a:
#     after.setPalette("RED-lfusion")
# w = a.createWindow('Sagittal')
# w.addObjects(list_after_a)
# w.addObjects(spam)

Position : 124, 153.975, 106.278, 0
